# Basic Configuration

Mysql is a leading open-source, multi-user and multi-threaded database management system. In this document, I try to figure out how to create, query, insert and terminate a mysql database based on c++ programming. The basic working environment is as below: 
- Gcc version 5.4.0 20160609 (Ubuntu 5.4.0-6ubuntu1~16.04.4) 
- Linux * 4.4.0-66-generic #87-Ubuntu x86_64 /Linux
- Server version: 5.7.17-0ubuntu0.16.04.1 (Ubuntu)

# How to Import and Export Databases
## Import a database

To import a database into mysql server, a new empty database can be created ahead in the MySQL shell to serve as a destination for imported database.
```
CREATE DATABASE newdatabase;
```
Then log out of the MySQL shell and type the following on the command line:
```
mysql -u [username] -p newdatabase < [database name].sql
```
With that, your chosen database will be imported into the specified database in MySQL.

## Export a database

To Export a database, open up terminal, making sure that you are not logged into MySQL and type,
```
mysqldump -u [username] -p [database name] > [database name].sql
```
The database that you selected will be exported to the current working directory.

## Start and Stop mysql service
```
/etc/init.d/mysql start
/etc/init.d/mysql stop
```

# Mysql Database Operations
In this use case, I tried to test the basic operations in database: create, insert and query a table

## Create a Database
Here a test database, named testdb is created for the coming test.
```c++
#include <my_global.h>
#include <mysql.h>

int main(int argc, char **argv)
{  
  // allocate or initialize a MYSQL object suitable for mysql_real_connect()
  MYSQL *con = mysql_init(NULL);

  if (con == NULL) 
  {
      fprintf(stderr, "%s\n", mysql_error(con));
      exit(1);
  }
  
  // Parameters: conn handler, hostname, username, password, db name, port number, unix socket and client flag
  if (mysql_real_connect(con, "localhost", "root", "root_pswd", NULL, 0, NULL, 0) == NULL) 
  {
      fprintf(stderr, "%s\n", mysql_error(con));
      mysql_close(con);
      exit(1);
  }  
  // Create test database
  if (mysql_query(con, "CREATE DATABASE testdb")) 
  {
      fprintf(stderr, "%s\n", mysql_error(con));
      mysql_close(con);
      exit(1);
  }

  mysql_close(con);
  exit(0);
}
```

Two errors occured in debuging:

Error 1
- fatal error: my_global.h: No such file or directory compilation terminated.

Solution:
```
cp -r /usr/include/mysql/* /usr/include/
```
Error 2
- fatal error: mysql.h: No such file or directory compilation terminated.

Solution:
```
/tmp/ccSTkxP3.o: In function `main':
database.c:(.text+0x15): undefined reference to `mysql_init'
database.c:(.text+0x2c): undefined reference to `mysql_error'
database.c:(.text+0x7d): undefined reference to `mysql_real_connect'
database.c:(.text+0x92): undefined reference to `mysql_error'
database.c:(.text+0xba): undefined reference to `mysql_close'
database.c:(.text+0xd5): undefined reference to `mysql_query'
database.c:(.text+0xe5): undefined reference to `mysql_error'
database.c:(.text+0x10d): undefined reference to `mysql_close'
database.c:(.text+0x123): undefined reference to `mysql_close'
```

- Compilation

```
g++ database.c -o database_cpp `mysql_config --cflags --libs`
g++ database.cpp -o database_cpp `mysql_config --cflags --libs`

gcc database.c -o database_c  -L/usr/include/mysql -lmysqlclient  
gcc database.cpp -o database_cpp  -L/usr/include/mysql -lmysqlclient  
```


## Create a Table
Create a new table, named Cars, in the newly created database.

```c++
#include <my_global.h>
#include <mysql.h>

void finish_with_error(MYSQL *con)
{
  fprintf(stderr, "%s\n", mysql_error(con));
  mysql_close(con);
  exit(1);        
}

int main(int argc, char **argv)
{
  MYSQL *con = mysql_init(NULL);
  
  if (con == NULL) 
  {
      fprintf(stderr, "%s\n", mysql_error(con));
      exit(1);
  }  

  //Pass the database name argument
  if (mysql_real_connect(con, "localhost", "root", "4259", "testdb", 0, NULL, 0) == NULL) 
  {
      finish_with_error(con);
  }    
  
  // Drop table Cars if exists
  if (mysql_query(con, "DROP TABLE IF EXISTS Cars")) {
      finish_with_error(con);
  }
  // Create table Cars
  if (mysql_query(con, "CREATE TABLE Cars(Id INT, Name TEXT, Price INT)")) {      
      finish_with_error(con);
  }
  // Insert values into table
  if (mysql_query(con, "INSERT INTO Cars VALUES(1,'Audi',52642)")) {
      finish_with_error(con);
  }
  
  if (mysql_query(con, "INSERT INTO Cars VALUES(2,'Mercedes',57127)")) {
      finish_with_error(con);
  }
  
  if (mysql_query(con, "INSERT INTO Cars VALUES(3,'Skoda',9000)")) {
      finish_with_error(con);
  }
  
  if (mysql_query(con, "INSERT INTO Cars VALUES(4,'Volvo',29000)")) {
      finish_with_error(con);
  }
  
  if (mysql_query(con, "INSERT INTO Cars VALUES(5,'Bentley',350000)")) {
      finish_with_error(con);
  }
  
  if (mysql_query(con, "INSERT INTO Cars VALUES(6,'Citroen',21000)")) {
      finish_with_error(con);
  }
  
  if (mysql_query(con, "INSERT INTO Cars VALUES(7,'Hummer',41400)")) {
      finish_with_error(con);
  }
  // Delete values
  if (mysql_query(con, "DELETE FROM Cars WHERE Id=6")) {
      finish_with_error(con);
  }
  // Query table through name
  if (mysql_query(con, "DELETE FROM Cars WHERE Name='Hummer'")) {
      finish_with_error(con);
  }

  mysql_close(con);
  exit(0);
}
```

## Query Opeartion
Query the table just created.

```c++
#include <my_global.h>
#include <mysql.h>

void finish_with_error(MYSQL *con)
{
  fprintf(stderr, "%s\n", mysql_error(con));
  mysql_close(con);
  exit(1);        
}

int main(int argc, char **argv)
{      
  MYSQL *con = mysql_init(NULL);
  
  if (con == NULL)
  {
      fprintf(stderr, "mysql_init() failed\n");
      exit(1);
  }  
  
  if (mysql_real_connect(con, "localhost", "root", "4259", "testdb", 0, NULL, 0) == NULL) 
  {
      finish_with_error(con);
  }    
  // query table Cars
  if (mysql_query(con, "SELECT * FROM Cars")) 
  {
      finish_with_error(con);
  }
  // Store query result
  MYSQL_RES *result = mysql_store_result(con);
  
  if (result == NULL) 
  {
      finish_with_error(con);
  }
  // Count fields
  int num_fields = mysql_num_fields(result);

  MYSQL_ROW row;
  // Loop all rows in query result, return NULL if there is no more row result
  while ((row = mysql_fetch_row(result))) 
  { 
      for(int i = 0; i < num_fields; i++) 
      { 
          printf("%s ", row[i] ? row[i] : "NULL"); 
      } 
          printf("\n"); 
  }
  
  mysql_free_result(result);
  mysql_close(con);
  
  exit(0);
}
```

## Insertion(Manually or automatically set ID)
This example shows how to automatically set primary key.

```c++
#include <my_global.h>
#include <mysql.h>

void finish_with_error(MYSQL *con)
{
  fprintf(stderr, "%s\n", mysql_error(con));
  mysql_close(con);
  exit(1);        
}

int main(int argc, char **argv)
{
  
  MYSQL *con = mysql_init(NULL);
  
  if (con == NULL)
  {
      fprintf(stderr, "mysql_init() failed\n");
      exit(1);
  }
  
  if (mysql_real_connect(con, "localhost", "root", "4259", "testdb", 0, NULL, 0) == NULL) 
  {
      finish_with_error(con);
  }   
  
  if (mysql_query(con, "DROP TABLE IF EXISTS Writers"))
  {    
      finish_with_error(con);    
  }
  
  char *sql = "CREATE TABLE Writers(Id INT PRIMARY KEY AUTO_INCREMENT, Name TEXT)";
  
  if (mysql_query(con, sql))
  {    
      finish_with_error(con);    
  }
  
  //Automatically set ID
  if (mysql_query(con, "INSERT INTO Writers(Name) VALUES('Leo Tolstoy')"))
  {    
      finish_with_error(con);    
  }
  
  // Manually set ID
  if (mysql_query(con, "INSERT INTO Writers VALUES(6,'Jack London')"))
  {    
      finish_with_error(con);
  }
  //Automatically set ID
  if (mysql_query(con, "INSERT INTO Writers(Name) VALUES('Honore de Balzac')"))
  {    
      finish_with_error(con);
  }
  
  int id = mysql_insert_id(con);
  
  printf("The last inserted row id is: %d\n", id);

  mysql_close(con);
  exit(0);
}
```

## Column Headers(Attributes of a table)
Query table and list all fields of table Cars.
```c++
#include <my_global.h>
#include <mysql.h>

void finish_with_error(MYSQL *con)
{
  fprintf(stderr, "%s\n", mysql_error(con));
  mysql_close(con);
  exit(1);        
}

int main(int argc, char **argv)
{
  MYSQL *con = mysql_init(NULL);
  
  if (con == NULL)
  {
      fprintf(stderr, "mysql_init() failed\n");
      exit(1);
  }  
  
  if (mysql_real_connect(con, "localhost", "root", "4259", "testdb", 0, NULL, 0) == NULL) 
  {
      finish_with_error(con);
  } 
  
  if (mysql_query(con, "SELECT * FROM Cars LIMIT 3"))
  {  
      finish_with_error(con);
  }
  
  MYSQL_RES *result = mysql_store_result(con);

  if (result == NULL) 
  {
      finish_with_error(con);
  }  

  int num_fields = mysql_num_fields(result);

  MYSQL_ROW row;
  MYSQL_FIELD *field;
  
  while ((row = mysql_fetch_row(result))) 
  { 
      for(int i = 0; i < num_fields; i++) 
      { 
          // Print attributes before print data
          if (i == 0) 
          {              
             while(field = mysql_fetch_field(result)) 
             {
                printf("%s ", field->name);
             }
             
             printf("\n");           
          }
          
          printf("%s  ", row[i] ? row[i] : "NULL"); 
      } 
  }
  
  printf("\n");
  
  mysql_free_result(result);
  mysql_close(con);
  
  exit(0);
}
```

## Multiple Statements
Test the function of multiple statements in a mysql_query().

```c++
#include <my_global.h>
#include <mysql.h>

void finish_with_error(MYSQL *con)
{
  fprintf(stderr, "%s\n", mysql_error(con));
  mysql_close(con);
  exit(1);        
}

int main(int argc, char **argv)
{ 
  int status = 0;  
    
  MYSQL *con = mysql_init(NULL);  
  
  if (con == NULL)
  {
      fprintf(stderr, "mysql_init() failed\n");
      exit(1);
  }  
  
  if (mysql_real_connect(con, "localhost", "root", "4259", "testdb", 0, NULL, CLIENT_MULTI_STATEMENTS) == NULL) 
  {
      finish_with_error(con);
  }    
  
  if (mysql_query(con, "SELECT Name FROM Cars WHERE Id=2;INSERT INTO Cars VALUES(7,'Hummer', 41400);\
          SELECT Name FROM Cars WHERE Name='audi';DELETE FROM Cars WHERE Id=5;")) 
  {
      finish_with_error(con);
  }
  
  do {  
      MYSQL_RES *result = mysql_store_result(con);
        
      if (result == NULL) 
      {
          finish_with_error(con);
      }
            
      MYSQL_ROW row = mysql_fetch_row(result);
      
      printf("%s\n", row[0]);
      
      mysql_free_result(result);
                 
      status = mysql_next_result(con); 
     
      if (status > 0) {
          finish_with_error(con);
      }
      
  } while(status == 0);
    
  mysql_close(con);  
  exit(0);
}
```

## Insert a Image
Try to figure out how to insert a image into database.
```c++
#include <my_global.h>
#include <mysql.h>
#include <string.h>

void finish_with_error(MYSQL *con)
{
  fprintf(stderr, "%s\n", mysql_error(con));
  mysql_close(con);
  exit(1);        
}

int main(int argc, char **argv)
{

  FILE *fp = fopen("lena.jpg", "rb");
  
  if (fp == NULL) 
  {
      fprintf(stderr, "cannot open image file\n");    
      exit(1);
  }
  // Set fp to the end of the pointed image
  fseek(fp, 0, SEEK_END);
  
  if (ferror(fp)) {
      
      fprintf(stderr, "fseek() failed\n");
      int r = fclose(fp);

      if (r == EOF) {
          fprintf(stderr, "cannot close file handler\n");          
      }    
      
      exit(1);
  }  
  // Record the size of the image
  int flen = ftell(fp);
  
  if (flen == -1) {
      
      perror("error occurred");
      int r = fclose(fp);

      if (r == EOF) {
          fprintf(stderr, "cannot close file handler\n");
      }
      
      exit(1);      
  }
  // Set fp to the begining of the pointed image
  fseek(fp, 0, SEEK_SET);
  
  if (ferror(fp)) {
      
      fprintf(stderr, "fseek() failed\n");
      int r = fclose(fp);

      if (r == EOF) {
          fprintf(stderr, "cannot close file handler\n");
      }    
      
      exit(1);
  }

  char data[flen+1];
  //Store image into a char array 
  int size = fread(data, 1, flen, fp);
  
  if (ferror(fp)) {
      
      fprintf(stderr, "fread() failed\n");
      int r = fclose(fp);

      if (r == EOF) {
          fprintf(stderr, "cannot close file handler\n");
      }
      
      exit(1);      
  }
  
  int r = fclose(fp);

  if (r == EOF) {
      fprintf(stderr, "cannot close file handler\n");
  }          
  
  MYSQL *con = mysql_init(NULL);
  
  if (con == NULL)
  {
      fprintf(stderr, "mysql_init() failed\n");
      exit(1);
  }  
    
  if (mysql_real_connect(con, "localhost", "root", "4259", 
          "testdb", 0, NULL, 0) == NULL) 
  {
      finish_with_error(con);
  }   
  //Crate a table to store images, mysql database has special data type to store image called BLOB 
  if (mysql_query(con,"CREATE TABLE Images(Id INT PRIMARY KEY, Data MEDIUMBLOB)"))
  {
      finish_with_error(con);
  }
   
  char chunk[2*size+1];
  // Convert into a legal mysql string for use in an SQL statement 
  mysql_real_escape_string(con, chunk, data, size);

  char *st = "INSERT INTO Images(Id, Data) VALUES(1, '%s')";
  size_t st_len = strlen(st);

  char query[st_len + 2*size+1]; 
  // Composes a string with the same text that would be printed if format was used on printf, but instead of being printed, the content is stored as a C string in the buffer pointed by s
  int len = snprintf(query, st_len + 2*size+1, st, chunk);

  if (mysql_real_query(con, query, len))
  {
      finish_with_error(con);
  }
  
  mysql_close(con);
  exit(0);
}
```

## Query a Image
Query the image just inserted
```c++
#include <my_global.h>
#include <mysql.h>

void finish_with_error(MYSQL *con)
{
  fprintf(stderr, "%s\n", mysql_error(con));
  mysql_close(con);
  exit(1);        
}

int main(int argc, char **argv)
{
  FILE *fp = fopen("lena_copy.jpg", "wb");
  
  if (fp == NULL) 
  {
      fprintf(stderr, "cannot open image file\n");    
      exit(1);
  }

  MYSQL *con = mysql_init(NULL);
  
  if (con == NULL)
  {
      fprintf(stderr, "mysql_init() failed\n");
      exit(1);
  }  
  
  if (mysql_real_connect(con, "localhost", "root", "4259", "testdb", 0, NULL, 0) == NULL) 
  {
      finish_with_error(con);
  }
  // Cannot query the image by Name
  if (mysql_query(con, "SELECT Data FROM Images WHERE Id=1"))
  {
      finish_with_error(con);
  }
  // Store query result, can use mysql_next_result() if there are multiple statements in the SQL command
  MYSQL_RES *result = mysql_store_result(con);
  
  if (result == NULL) 
  {
      finish_with_error(con);
  }  
  //fetch first row of the result
  MYSQL_ROW row = mysql_fetch_row(result);
  //Size of the data in current row, maybe contain many fields
  unsigned long *lengths = mysql_fetch_lengths(result);
  
  if (lengths == NULL) {
      finish_with_error(con);
  }
  //Write the queried image into fp
  fwrite(row[0], lengths[0], 1, fp);

  if (ferror(fp)) 
  {            
      fprintf(stderr, "fwrite() failed\n");
      mysql_free_result(result);
      mysql_close(con);

      exit(1);      
  }  
  
  int r = fclose(fp);

  if (r == EOF) {
      fprintf(stderr, "cannot close file handler\n");
  }
  
  mysql_free_result(result);
  mysql_close(con);

  exit(0);
}
```

# References
- MySQL C API programming tutorial: http://zetcode.com/db/mysqlc/
- MySQL Official Document: https://dev.mysql.com/doc/refman/5.7/en/